### IND

+ `admin_name1` = `region`
+ `admin_name2` = `city`
+ `admin_name3` = `city`



In [11]:
import pandas as pd
import warnings
from unidecode import unidecode

warnings.filterwarnings("ignore")

**Crunchbase Dataframe**

In [12]:
df = pd.read_csv('input/foodtech.csv')
df = df[df['country_code'] == 'IND']
df = df[['uuid','country_code', 'state_code', 'region', 'city', 'address', 'postal_code']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8141 entries, 2 to 163640
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uuid          8141 non-null   object
 1   country_code  8141 non-null   object
 2   state_code    0 non-null      object
 3   region        8141 non-null   object
 4   city          8141 non-null   object
 5   address       5416 non-null   object
 6   postal_code   5492 non-null   object
dtypes: object(7)
memory usage: 508.8+ KB


*Reformatting*

In [13]:
df['postal_code'] = df['postal_code'].str.replace(' ', '')

**Postal Codes**

In [14]:
codes = pd.read_json('input/geonames.json')
codes = codes[codes['country_code'] == 'IN']

`admin_name3`

In [15]:
codes1 = codes.copy()
codes1 = codes1[['postal_code','admin_name1', 'admin_name3']]
codes1.drop_duplicates(subset=['postal_code','admin_name1', 'admin_name3'], inplace=True)


In [16]:
merged_df = pd.merge(df, codes1, left_on=['city', 'region'], right_on=['admin_name3', 'admin_name1'], how='left')
merged_df.drop_duplicates(subset=['uuid'], inplace=True)

In [17]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8141 entries, 0 to 171498
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           8141 non-null   object
 1   country_code   8141 non-null   object
 2   state_code     0 non-null      object
 3   region         8141 non-null   object
 4   city           8141 non-null   object
 5   address        5416 non-null   object
 6   postal_code_x  5492 non-null   object
 7   postal_code_y  6855 non-null   object
 8   admin_name1    6855 non-null   object
 9   admin_name3    6855 non-null   object
dtypes: object(10)
memory usage: 699.6+ KB


`admin_name2`

In [18]:
codes2 = codes.copy()
codes2 = codes2[['postal_code','admin_name1', 'admin_name2']]

codes2['admin_name2'] = codes2['admin_name2'].str.replace('Bangalore Rural', 'Bangalore')
codes2['admin_name2'] = codes2['admin_name2'].str.replace('Ananthapur', 'Anantapur')
codes2['admin_name2'] = codes2['admin_name2'].str.replace('Darjiling', 'Darjeeling')

codes2.loc[codes2['admin_name2'] == 'Hyderabad', 'admin_name1'] = codes2['admin_name1'].str.replace('Telangana', 'Andhra Pradesh')
codes2.loc[(codes2['admin_name1'] == 'Delhi') & (codes2['admin_name2'].str.contains('Delhi') & ~codes2['admin_name2'].str.contains('New Delhi')), 'admin_name2'] = 'Delhi'

codes2.drop_duplicates(subset=['admin_name1', 'admin_name2'], inplace=True)


In [19]:
merged_df = pd.merge(merged_df, codes2, left_on=['city', 'region'], right_on=['admin_name2', 'admin_name1'], how='left')
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8141 entries, 0 to 8140
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           8141 non-null   object
 1   country_code   8141 non-null   object
 2   state_code     0 non-null      object
 3   region         8141 non-null   object
 4   city           8141 non-null   object
 5   address        5416 non-null   object
 6   postal_code_x  5492 non-null   object
 7   postal_code_y  6855 non-null   object
 8   admin_name1_x  6855 non-null   object
 9   admin_name3    6855 non-null   object
 10  postal_code    6483 non-null   object
 11  admin_name1_y  6483 non-null   object
 12  admin_name2    6483 non-null   object
dtypes: object(13)
memory usage: 890.4+ KB


**Filling NaNs with Existing Postal Codes**

In [20]:
merged_df['postal_code_y'].fillna(merged_df['postal_code'], inplace=True)
merged_df['postal_code_y'].fillna(merged_df['postal_code_x'], inplace=True)

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8141 entries, 0 to 8140
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           8141 non-null   object
 1   country_code   8141 non-null   object
 2   state_code     0 non-null      object
 3   region         8141 non-null   object
 4   city           8141 non-null   object
 5   address        5416 non-null   object
 6   postal_code_x  5492 non-null   object
 7   postal_code_y  7931 non-null   object
 8   admin_name1_x  6855 non-null   object
 9   admin_name3    6855 non-null   object
 10  postal_code    6483 non-null   object
 11  admin_name1_y  6483 non-null   object
 12  admin_name2    6483 non-null   object
dtypes: object(13)
memory usage: 890.4+ KB


*Across rows: df*

In [21]:
merged_df = merged_df[merged_df.columns[:-5]]

In [22]:
codes_df = df[df['postal_code'].notna()]

codes_df = codes_df[['region', 'city', 'postal_code']]
codes_df.drop_duplicates(subset=['region', 'city'], inplace=True)

In [23]:
merged_df = pd.merge(merged_df, codes_df, left_on=['city', 'region'], right_on=['city', 'region'], how='left')
merged_df['postal_code_y'].fillna(merged_df['postal_code'], inplace=True)

In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8141 entries, 0 to 8140
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           8141 non-null   object
 1   country_code   8141 non-null   object
 2   state_code     0 non-null      object
 3   region         8141 non-null   object
 4   city           8141 non-null   object
 5   address        5416 non-null   object
 6   postal_code_x  5492 non-null   object
 7   postal_code_y  8088 non-null   object
 8   postal_code    8020 non-null   object
dtypes: object(9)
memory usage: 636.0+ KB


*Checking NaNs*

In [25]:
nan_df = merged_df[merged_df['postal_code_y'].isna()]
nan_df['region'].value_counts()

West Bengal                    7
Tamil Nadu                     5
Maharashtra                    5
Uttar Pradesh                  4
Kerala                         4
Goa                            4
Orissa                         3
Chhattisgarh                   3
Jammu and Kashmir              3
Delhi                          2
Manipur                        2
Daman and Diu                  2
Andaman and Nicobar Islands    1
Madhya Pradesh                 1
Assam                          1
Rajasthan                      1
Andhra Pradesh                 1
Karnataka                      1
Arunachal Pradesh              1
Bihar                          1
Haryana                        1
Name: region, dtype: int64

*Save to csv*

In [26]:
merged_df = merged_df.rename(columns={'postal_code_x': 'pc_crunchbase', 
                                      'postal_code_y': 'pc_filled'})

merged_df = merged_df[['uuid', 'pc_crunchbase', 'pc_filled']]


In [27]:
merged_df.to_csv('processed/IND_processed.csv')